# Note:
I used Kaggle notebook to run this notebook. The input data are pre-loaded, so I do not need to download.

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv


From the Discussions:
1. `weather==4` and `humidity==0` are outliers. I will remove them.
2. Beside `'casual'` and `'registered'`, `'month'` and `'datetime'` can also be dropped.
3. Try to use GridSearchCV for hyperparameter tuning.

I think that I can think of this after exploratory data analysis and experimenting. However, the competition Discussion often has many useful information that can save you the trouble of doing it yourself. Of course, you must make sure that the person was not lying first.

In [2]:
data_path = "/kaggle/input/bike-sharing-demand/"
train=pd.read_csv(data_path+'train.csv')
test=pd.read_csv(data_path+'test.csv')
submission=pd.read_csv(data_path+'sampleSubmission.csv')

In [3]:
train = train[(train['weather']!=4)&(train['humidity']!=0)]
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
all_data = pd.concat([train, test])
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0


In [5]:
all_data.tail()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
6488,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
6489,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
6490,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN
6491,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN
6492,2012-12-31 23:00:00,1,0,1,1,10.66,13.635,65,8.9981,NaN,NaN,NaN


In [6]:
all_data['datetime'] = pd.to_datetime(all_data['datetime'])
all_data['year'] = all_data['datetime'].dt.year
all_data['month'] = all_data['datetime'].dt.month
all_data['hour'] = all_data['datetime'].dt.hour

In [7]:
all_data.drop(columns=['casual', 'registered', 'month', 'datetime'], inplace=True)

In [8]:
train_length = len(train)

In [9]:
X_train = all_data[:train_length]
X_test = all_data[train_length:]
y = train['count']

In [10]:
X_train = X_train.drop(columns = ['count'], axis=1)
X_test = X_test.drop(columns = ['count'], axis=1)

In [11]:
print(X_train.shape, X_test.shape)

(10863, 10) (6493, 10)


In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.2, random_state=42)

The author [polljjaks](https://www.kaggle.com/polljjaks) used RMSLE (Root Mean Squared Logarithmic Error) as the scoring function for `GriSearchCV`. From my understanding, this scoring function is more effective compared to RMSE when the target variable has a wide range of values and/or has a skewed distribution. From [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter),it seems that `sklearn.model_selection.GridSearchCV` has an option for pre-defined MSLE, which can serve the same purpose. For the sake of reproducibility, let's keep the scoring function.

In [13]:
# Courtesy of the author
def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        # exponentialize the predicted and actual values obtained by log(count) and return them to the original count
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
     
    #Convert missing values to zero after logarithmic transformation
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    #Calculate RMSLE
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

The author used the logarithm of the target value instead of the actual values. It served the same purpose as RMSLE.

In [16]:
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

params = {'n_estimators': [100, 150, 200], 
              'max_depth': [3, 4, 5], 
              'learning_rate': [0.05, 0.1, 0.15]}

rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

gridsearch_model = GridSearchCV(estimator=model, 
                               param_grid = params,
                               scoring=rmsle_scorer,
                               cv=5)

log_y = np.log(y_train)

gridsearch_model.fit(X_train, log_y)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             scoring=make_scorer(rmsle, greater_is_better=False))

In [17]:
print(gridsearch_model.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [19]:
log_y2 = np.log(y_valid)
preds = gridsearch_model.best_estimator_.predict(X_valid)

print(f'RMSLE: {rmsle(log_y2, preds, True):.4f}')

RMSLE: 0.3190


In [20]:
test_preds = gridsearch_model.best_estimator_.predict(X_test)
submission['count'] = np.exp(test_preds)
submission.to_csv('submission.csv', index=False)

In [21]:
# A different way to create kaggle.json file
creds = ''

from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [22]:
!kaggle competitions submit -c bike-sharing-demand -f submission.csv -m "gradient boosting with GridSearchCV"

100%|████████████████████████████████████████| 243k/243k [00:00<00:00, 1.10MB/s]
Successfully submitted to Bike Sharing Demand

In [23]:
!kaggle competitions submissions -c bike-sharing-demand | tail -n +1 | head -n 6

fileName                     date                 description                             status    publicScore  privateScore  
---------------------------  -------------------  --------------------------------------  --------  -----------  ------------  
submission.csv               2023-05-27 02:17:27  gradient boosting with GridSearchCV     complete  0.38863      0.38863       
submission_new_hpo_1.csv     2023-05-26 08:41:56  new features with increased time limit  complete  0.67834      0.67834       
submission_new_hpo.csv       2023-05-26 08:19:14  new features with hyperparameters       complete  0.61537      0.61537       
submission_new_features.csv  2023-05-26 08:06:23  new features                            complete  0.63415      0.63415       
